In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True
dj.config['display.limit'] = 100

name = 'simdata' #"simdata"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [17]:
import torch
import shutil
import numpy as np
import pickle 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
import matplotlib.pyplot as plt
import re
import seaborn as sns

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder
from nnfabrik.utility.hypersearch import Bayesian

from nnsysident.tables.experiments import *
from nnsysident.tables.bayesian import *
from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader

def find_number(text, c):
    return re.findall(r'%s(\d+)' % c, text)

def get_transfer(old_experiment_name):
    # prepare the Transfer table in a way that all the info about the transferred model is in the DataFrame. Just pd.merge (on transfer_fn and transfer_hash)
    # it then with the model that the transferred model was used for. 
    transfer = pd.DataFrame(Transfer.fetch())
    transfer = pd.concat([transfer, transfer['transfer_config'].apply(pd.Series)], axis = 1).drop('transfer_config', axis = 1)

    tm = pd.DataFrame((TrainedModel * Dataset * Seed * Experiments.Restrictions & 'experiment_name = "{}"'.format(old_experiment_name)).fetch()).rename(
        columns = {'model_hash': 't_model_hash', 'trainer_hash': 't_trainer_hash', 'dataset_hash': 't_dataset_hash'})
    tm = tm.sort_values('score', ascending=False).drop_duplicates(['t_model_hash', 't_trainer_hash', 't_dataset_hash'])

    transfer = pd.merge(transfer, tm, how='inner', on=['t_model_hash', 't_trainer_hash', 't_dataset_hash'])
    transfer = pd.concat([transfer, transfer['dataset_config'].apply(pd.Series)], axis = 1).drop('dataset_config', axis = 1)
    transfer.columns = ['t_' + col if col[:2] != 't_' and col[:8] != 'transfer'  else col for col in transfer.columns]
    transfer = transfer.sort_values(['t_neuron_n', 't_image_n', 't_neuron_base_seed', 't_image_base_seed'])
    return transfer

In [41]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__trained_model_transfer,0d7e6e1e9216be0c1c94b6074760a3b7,reserved,=BLOB=,,=BLOB=,konstantin@134.2.168.107,9efcd11270a5,1,65712,2020-05-13 18:27:24
__trained_model_transfer,9452af64a11127377b972998d5e2d74c,reserved,=BLOB=,,=BLOB=,konstantin@134.2.168.119,84b4c4fd5472,1,65710,2020-05-13 18:27:59
__trained_model_transfer,aead1a1712d21e3aa0345819e0ba42aa,reserved,=BLOB=,,=BLOB=,konstantin@134.2.168.107,475459e6c34b,1,65713,2020-05-13 18:27:33
__trained_model_transfer,b2a8a98fdab902240eaa182ef897c1b0,reserved,=BLOB=,,=BLOB=,konstantin@sinz00.mlcloud.uni-tuebingen.de,4fe96d35ee57,1,65699,2020-05-13 18:28:06
__trained_model_transfer,d81032f7539f56fb9cc67eff5c694166,reserved,=BLOB=,,=BLOB=,konstantin@134.2.168.119,56729d6cdd17,1,65711,2020-05-13 18:27:38
__trained_model_transfer,f18e3e75b5716d4832b14cd4e694d0bd,reserved,=BLOB=,,=BLOB=,konstantin@sinz00.mlcloud.uni-tuebingen.de,49667ea905c3,1,65697,2020-05-13 18:27:56


In [ ]:
experiment_name = 'SIM, Direct, se2d_spatialxfeaturelinear, 0-0-3'

TrainedModel.progress(Experiments.Restrictions & 'experiment_name="{}"'.format(experiment_name))

In [ ]:
experiment_name = 'SIM, Direct, se2d_pointpooled, 0-0-3'

TrainedModel.progress(Experiments.Restrictions & 'experiment_name="{}"'.format(experiment_name))

In [ ]:
experiment_name = 'SIM, Direct, se2d_fullgaussian2d, 0-0-3'

TrainedModel.progress(Experiments.Restrictions & 'experiment_name="{}"'.format(experiment_name))